In [ ]:
import sys
sys.path.append('../keras_test')

In [ ]:
batch_size = 128
n_classes = 10


In [ ]:
from load_mnist import load_mnist
(x_train, y_train), (x_test, y_test) = load_mnist(x_style='one_channel')


In [ ]:
x_train.shape

In [ ]:
epochs = 3
log_filepath = './log'


In [ ]:
from keras.models import Model, Sequential
from keras.engine.topology import Input
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam
import keras.callbacks
from tensorboard_environment import TensorboardEnvironment
from my_model import simple_full_connected, vgg_like
import os
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='0'

with TensorboardEnvironment():
    input_shape = x_train.shape[1:]
    input_tensor = Input(shape=input_shape)
    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(input_tensor)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    x = Flatten(name='flatten')(x)
    x = Dense(1024, activation='relu', name='fc1')(x)
    x = Dense(1024, activation='relu', name='fc2')(x)
    x = Dense(n_classes, activation='softmax', name='predictions')(x)
    model = Model(inputs=input_tensor, outputs=x)
    model.summary()
    optimizer = SGD(lr=0.01, momentum=0.1, decay=0.0, nesterov=False)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_filepath, histogram_freq=1)
    history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, callbacks=[tensorboard_callback], validation_split=0.1)
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy;', score[1])
